<img src="logo.png" width="40%">

Ontology based atomic structure creation, manipulation, querying.

Imports

In [1]:
from pyscal_rdf import StructureGraph

First we create a graph

In [2]:
g = StructureGraph()

## Creation of structures

We will create three structures for the demonstration.

First a BCC Iron structure

In [ ]:
struct_Fe = g.create_element("Fe")

Note that when the structure is created, it is also added to the Graph automatically. We can visualise the graph.

In [ ]:
g.visualise()

Now a Si diamond structure

In [ ]:
struct_Si = g.create_element("Si")

Finally, an L12 $Ni_3 Al$ structure

In [3]:
struct_l12 = g.create_structure("l12", element=['Al', 'Ni'], 
                         lattice_constant=3.57)

['Al', 'Ni']


In [4]:
g.visualise()

CytoscapeWidget(cytoscape_layout={'name': 'cola', 'avoidOverlap': True, 'animate': True}, cytoscape_style=[{'sâ€¦

We can save the graph and reload it as needed

## Querying the graph

An example question would be, **what are the space group of all structures with 4 atoms?**


The corresponding SPARQL query looks like this:

In [ ]:
query = """
PREFIX cmso: <https://purls.helmholtz-metadaten.de/cmso/>
SELECT DISTINCT ?symbol
WHERE {
    ?sample cmso:hasNumberOfAtoms ?number .
    ?sample cmso:hasMaterial ?material .
    ?material cmso:hasStructure ?structure .
    ?structure cmso:hasSpaceGroup ?spacegroup .
    ?spacegroup cmso:hasSpaceGroupSymbol ?symbol .
FILTER (?number="4"^^xsd:integer)
}"""

In [ ]:
res = g.graph.query(query)

And print the results

In [ ]:
for r in res:
    print(r[0])

There are also pre-built queries. For example, **What are all the samples with Bravais lattice bcc?**

In [ ]:
from pyscal_rdf.queries import Query

In [ ]:
q = Query()

In [ ]:
samples = q.sparql.sample_by_latticesystem(g, 'bcc')

We can check how many samples we found

In [ ]:
len(samples)

As expected, there is only one sample

We can write this sample to a file, for example, a LAMMPS data format, to use it for further simulations

In [ ]:
g.to_file(samples[0], 'bcc.data', format="lammps-data")

In [ ]:
! more bcc.data

We can also export as an ASE object

In [ ]:
aseobj = g.to_file(samples[0], format="ase")

In [ ]:
aseobj

## Grain boundaries

Still under active development (early stages)..

We start by creating a $\Sigma 5 (3 \bar{1} 0)$

In [ ]:
struct_gb_1 = g.create_grain_boundary(axis=[0,0,1], 
                        sigma=5, 
                        gb_plane=[3, -1, 0],
                        element='Fe')

Some other examples, $\Sigma 19 (\bar{3}20)$ and $\Sigma 19 (111)$

In [ ]:
struct_gb_2 = g.create_grain_boundary(axis=[1,1,1], 
                        sigma=19, 
                        gb_plane=[-3, 2, 0],
                        element='Fe')

struct_gb_3 = g.create_grain_boundary(axis=[1,1,1], 
                        sigma=19, 
                        gb_plane=[-1, -1, -1],
                        element='Fe')

**What are all the samples with symmetric tilt grain boundaries?**

In [ ]:
samples = q.sparql.sample_by_defect(g, "symmetric tilt")

In [ ]:
samples

In [ ]:
g.to_file(samples[0], filename="POSCAR", format="poscar")

In [ ]:
! head -20 POSCAR

**What are all the samples with boundaries with sigma values between 7 and 21?**

In [ ]:
samples = q.sparql.sample_by_sigma(g, [7, 21])

In [ ]:
len(samples)